In [ ]:
import gym
import numpy as np
from tqdm import tqdm
import torch
import wandb

from fractal_zero.config import FMCConfig
from fractal_zero.search.fmc import FMC
from fractal_zero.models.prediction import FullyConnectedPredictionModel
from fractal_zero.models.policies.cartpole_policy import CartpolePolicy
from fractal_zero.vectorized_environment import (
    RayVectorizedEnvironment,
    VectorizedDynamicsModelEnvironment,

)
from fractal_zero.trainers.online import OnlineFMCPolicyTrainer
from fractal_zero.tests.test_vectorized_environment import build_test_joint_model

In [ ]:
NUM_WALKERS = 64



policy_model = CartpolePolicy()

In [ ]:
# optimizer = torch.optim.SGD(policy_model.parameters(), lr=0.01, weight_decay=1e-4)
optimizer = torch.optim.Adam(policy_model.parameters(), lr=0.03, weight_decay=1e-3)
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)
policy_trainer = OnlineFMCPolicyTrainer("CartPole-v0", policy_model, optimizer, NUM_WALKERS)

In [ ]:
wandb.init(project="fz-policy-trainer-game-tree")

In [ ]:
from copy import deepcopy


num_episodes = 10
train_steps_per_episode = 100
eval_every = 20
max_steps = 200

best_total_rewards = float("-inf")
best_model = None

for _ in range(num_episodes):
    policy_trainer.generate_episode_data(max_steps)

    for i in range(train_steps_per_episode):
        policy_trainer.train_on_latest_episode()

        if i % eval_every == 0:
            total_rewards = policy_trainer.evaluate_policy(max_steps)

            if total_rewards > best_total_rewards:
                best_total_rewards = total_rewards
                best_model = deepcopy(policy_model)
                
                torch.save(best_model, "models/online_fmc.pth")

    # lr_scheduler.step()

In [ ]:
policy_trainer.fmc.clone_receives

In [ ]:
# policy_trainer.fmc.tree.render()

In [ ]:
str(policy_trainer.fmc.tree.g)

In [ ]:
# tree = policy_trainer.fmc.tree.g
# for node in tree:
#     print(node, tree.out_degree(node), node.visits)